In [24]:
import pandas as pd

from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

#предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, \
                                precision_recall_curve, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [3]:
# Загрузим пользователей и списки последних прочитанных новостей
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [4]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [34]:
#import nltk
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aermo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [6]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [7]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [8]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

<ipython-input-7-7ee348d9b386>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


Wall time: 45.9 s


In [9]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 6min 22s


In [10]:
news['title']

0        [заместитель, председатель, правительство, рф,...
1        [матч, финал, кубок, россия, футбол, приостано...
2        [форвард, авангард, томаш, заборский, прокомме...
3        [главный, тренер, кубань, юрий, красножанин, п...
4        [решение, попечительский, совет, владивостокск...
                               ...                        
26995    [учёный, токийский, университет, морской, наук...
26996    [глава, кафедра, отечественный, история, xx, в...
26997    [американский, учёный, уточнить, возраст, расп...
26998    [последний, год, тропический, углеродный, цикл...
26999    [жить, примерно, тыс, год, назад, территория, ...
Name: title, Length: 27000, dtype: object

In [11]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

### Запускаем обучение

In [12]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 1min


In [16]:
''' from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file) '''

' from gensim.test.utils import datapath\n# Save model to disk.\ntemp_file = datapath("model.lda")\nlda.save(temp_file)\n\n# Load a potentially pretrained model from disk.\nlda = LdaModel.load(temp_file) '

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [13]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(10, 0.08610183),
 (14, 0.14561965),
 (17, 0.05312407),
 (22, 0.38716102),
 (23, 0.31021854)]

In [14]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: млн тыс исследование год составить турция турецкий
topic_1: температура градус умереть билет лагерь палатка страдать
topic_2: земля это армия nn мочь который мозг
topic_3: год компания это млрд рынок который цена
topic_4: год который человек это жизнь nn дом
topic_5: двигатель солнце эксперимент сигнал погода дождь подросток
topic_6: это который россия мочь украина год весь
topic_7: рт пролететь корь аргумент выдать герман волков
topic_8: статья мвд ст регистрация товарищ лесной отдел
topic_9: взрыв пострадать террорист озеро констатировать боевик сталкиваться
topic_10: превысить климат потребность производить горизонт оплата багаж
topic_11: место район пациент результат nn экипаж исследование
topic_12: военный сша американский миссия который поверхность это
topic_13: исследование который год расследование британский торговый произойти
topic_14: год это который свой человек весь писать
topic_15: фестиваль женщина девочка треть польша мужчина германия
topic_16: россия это nn аг

### Напишем функцию, которая будет нам возвращать векторное представление новости

In [15]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [16]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.056285,0.865124,0.070488
1,4896,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.337929,0.340295,0.000000
2,4897,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.053437,0.0,0.000000,0.0,0.0,0.387084,0.310223,0.000000
3,4898,0.024546,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.646875,0.142373,0.0,0.000000,0.0,0.0,0.176252,0.000000,0.000000
4,4899,0.000000,0.0,0.186401,0.0,0.0,0.0,0.048407,0.0,0.0,...,0.0,0.000000,0.071475,0.0,0.278568,0.0,0.0,0.105381,0.288549,0.000000


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

### Следующий шаг - векторные представления пользователей

In [17]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [18]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [19]:
doc_dict[2]

array([0.        , 0.        , 0.        , 0.08496894, 0.        ,
       0.        , 0.09057752, 0.        , 0.        , 0.05739897,
       0.        , 0.        , 0.16353877, 0.        , 0.        ,
       0.        , 0.14623874, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.4399794 , 0.        ])

In [20]:
user_articles_list = users['articles'].iloc[33]

# Модифицируем функцию
def get_user_embedding(user_articles_list, metric=np.mean):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = metric(user_vector, 0)
    return user_vector

In [21]:
get_user_embedding(user_articles_list)

array([0.004736  , 0.02420798, 0.03114678, 0.07366884, 0.06533103,
       0.0157472 , 0.22126689, 0.        , 0.        , 0.        ,
       0.        , 0.00644295, 0.0336629 , 0.03012838, 0.05700939,
       0.        , 0.05654988, 0.        , 0.        , 0.06953661,
       0.        , 0.05280095, 0.00508914, 0.14881853, 0.08886101])

In [22]:
# Функция расчета метрик
def get_metrics(metric):
    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, metric), 1)])
    user_embeddings.columns = [f'topic_{i}' for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(25)]]
    X = pd.merge(user_embeddings, target, 'left')

    X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(25)]], 
                                                        X['churn'], random_state=0)

    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    preds = logreg.predict_proba(X_test)[:, 1]

    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    roc_auc = roc_auc_score(y_test, preds)
    ix = np.argmax(fscore)
    
    return roc_auc, precision[ix], recall[ix], fscore[ix]

mean

In [25]:
m = list(get_metrics(np.mean))
print(f'roc_auc = {m[0]:.3f}, precision = {m[1]:.3f}, recall = {m[2]:.3f}, fscore = {m[3]:.3f}')

roc_auc = 0.931, precision = 0.516, recall = 0.837, fscore = 0.639


median

In [26]:
m = list(get_metrics(np.median))
print(f'roc_auc = {m[0]:.3f}, precision = {m[1]:.3f}, recall = {m[2]:.3f}, fscore = {m[3]:.3f}')

roc_auc = 0.966, precision = 0.636, recall = 0.914, fscore = 0.750


max

In [27]:
m = list(get_metrics(np.max))
print(f'roc_auc = {m[0]:.3f}, precision = {m[1]:.3f}, recall = {m[2]:.3f}, fscore = {m[3]:.3f}')

roc_auc = 0.975, precision = 0.758, recall = 0.816, fscore = 0.786


### Сформируем таблицу

In [29]:
metrics = {'mean': np.mean, 'median': np.median, 'max': np.max}
info = pd.DataFrame(columns=metrics.keys())

In [30]:
for metric, value in metrics.items():
    print(metric, value)
    roc_auc, precision, recall, fscore = get_metrics(value)
    info.loc[f'roc_auc', metric] = roc_auc
    info.loc[f'precision', metric] = precision
    info.loc[f'recall', metric] = recall
    info.loc[f'fscore', metric] = fscore
    
info

mean <function mean at 0x000001CE7CF76670>
median <function median at 0x000001CE7D0A41F0>
max <function amax at 0x000001CE7CF718B0>


,mean,median,max
roc_auc,0.931003,0.965756,0.97458
precision,0.516373,0.636364,0.757576
recall,0.836735,0.914286,0.816327
fscore,0.638629,0.750419,0.785855


Если берём медиану или мах, то получаем более лучшие результаты. Если нам важен recall то берем медиану, если precision то max